# Overview

This notebook contains my solutions for **<a href="https://adventofcode.com/2017" target="_blank">Advent of Code 2017</a>**.

A few notes...
- The source for this notebook source lives in my GitHub repo, <a href="https://github.com/derailed-dash/Advent-of-Code/blob/master/src/AoC_2017/Dazbo's_Advent_of_Code_2017.ipynb" target="_blank">here</a>.
- You can run this Notebook wherever you like. For example, you could...
  - Run it locally, in your own Jupyter environment.
  - Run it in a cloud-based Jupyter environment, with no setup required on your part!  For example, <a href="https://colab.research.google.com/github/derailed-dash/Advent-of-Code/blob/master/src/AoC_2017/Dazbo's_Advent_of_Code_2017.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Google Colab"/></a>
- All of my AoC solutions are documented in my <a href="https://aoc.just2good.co.uk/" target="_blank">AoC Python Walkthrough site</a>.
- To run the notebook, execute the cells in the [Setup](#Setup) section, as described below. Then you can run the code for any given day.

# Setup

You need to run all cells in this section, before running any particular day solution.

## Packages and Imports

Here we use `pip` to install the packages used by my solutions in this event.

In [ ]:
%pip install jupyterlab-lsp
%pip install colorama

In [ ]:
from __future__ import annotations
import copy
from dataclasses import asdict, dataclass
from enum import Enum
from functools import cache
from itertools import permutations, combinations
import operator
import logging
import time
import os
import re
import unittest
import requests
import numpy as np
from pathlib import Path
from getpass import getpass
from colorama import Fore
from IPython.display import display, Markdown

## Get Access to Your AoC Data

Now provide your unique AoC session key, in order to download your input data. You can get this by:
1. Logging into [Advent of Code](https://adventofcode.com/).
1. From your browser, open Developer Tools. (In Chrome, you can do this by pressing F12.)
1. Open the `Application` tab.
1. Storage -> Cookies -> https://adventofcode
1. Copy the value associated with the cookie called `session`.

![Finding the session cookie](https://aoc.just2good.co.uk/assets/images/aoc-cookie.png)




In [ ]:
if not os.getenv('AOC_SESSION_COOKIE'):
    os.environ['AOC_SESSION_COOKIE'] = getpass('Enter AoC session key: ')

## Load Helpers and Useful Classes

Now we load a bunch of helper functions and classes.

### Logging

Set up a new logger that uses `ColouredFormatter`, such that we have coloured logging.  The log colour depends on the logging level.

In [ ]:
##########################################################################
# SETUP LOGGING
#
# Create a new instance of "logger" in the client application
# Set to your preferred logging level
# And add the stream_handler from this module, if you want coloured output
##########################################################################

# logger for aoc_commons only
logger = logging.getLogger(__name__) # aoc_common.aoc_commons
logger.setLevel(logging.INFO)
stream_handler = None

class ColouredFormatter(logging.Formatter):
    """ Custom Formater which adds colour to output, based on logging level """

    level_mapping = {"DEBUG": (Fore.BLUE, "DBG"),
                     "INFO": (Fore.GREEN, "INF"),
                     "WARNING": (Fore.YELLOW, "WRN"),
                     "ERROR": (Fore.RED, "ERR"),
                     "CRITICAL": (Fore.MAGENTA, "CRT")
    }

    def __init__(self, *args, apply_colour=True, shorten_lvl=True, **kwargs) -> None:
        """ Args:
            apply_colour (bool, optional): Apply colouring to messages. Defaults to True.
            shorten_lvl (bool, optional): Shorten level names to 3 chars. Defaults to True.
        """
        super().__init__(*args, **kwargs)
        self._apply_colour = apply_colour
        self._shorten_lvl = shorten_lvl

    def format(self, record):
        if record.levelname in ColouredFormatter.level_mapping:
            new_rec = copy.copy(record)
            colour, new_level = ColouredFormatter.level_mapping[record.levelname]

            if self._shorten_lvl:
                new_rec.levelname = new_level

            if self._apply_colour:
                msg = colour + super().format(new_rec) + Fore.RESET
            else:
                msg = super().format(new_rec)

            return msg

        # If our logging message is not using one of these levels...
        return super().format(record)

if not stream_handler:
    stream_handler = logging.StreamHandler()
    stream_fmt = ColouredFormatter(fmt='%(asctime)s.%(msecs)03d:%(name)s - %(levelname)s: %(message)s',
                                   datefmt='%H:%M:%S')
    stream_handler.setFormatter(stream_fmt)
    
if not logger.handlers:
    # Add our ColouredFormatter as the default console logging
    logger.addHandler(stream_handler)

def retrieve_console_logger(script_name):
    """ Create and return a new logger, named after the script
    So, in your calling code, add a line like this:
    logger = ac.retrieve_console_logger(locations.script_name)
    """
    a_logger = logging.getLogger(script_name)
    a_logger.addHandler(stream_handler)
    a_logger.propagate = False
    return a_logger

def setup_file_logging(a_logger: logging.Logger, folder: str|Path=""):
    """ Add a FileHandler to the specified logger. File name is based on the logger name.
    In calling code, we can add a line like this:
    td.setup_file_logging(logger, locations.output_dir)

    Args:
        a_logger (Logger): The existing logger
        folder (str): Where the log file will be created. Will be created if it doesn't exist
    """
    Path(folder).mkdir(parents=True, exist_ok=True)     # Create directory if it does not exist
    file_handler = logging.FileHandler(Path(folder, a_logger.name + ".log"), mode='w')
    file_fmt = logging.Formatter(fmt="%(asctime)s.%(msecs)03d:%(name)s:%(levelname)8s: %(message)s",
                                datefmt='%H:%M:%S')
    file_handler.setFormatter(file_fmt)
    a_logger.addHandler(file_handler)

### Locations

Where any input and output files get stored.

<img src="https://aoc.just2good.co.uk/assets/images/notebook-content-screenshot.png" width="320" />


In [ ]:
#################################################################
# Paths and Locations
#################################################################

@dataclass
class Locations:
    """ Dataclass for storing various location properties """
    script_name: str
    script_dir: Path
    input_dir: Path
    output_dir: Path
    input_file: Path

def get_locations(script_name, folder="") -> Locations:
    """ Set various paths, based on the location of the calling script. """
    current_directory = os.getcwd()
    script_dir = Path(Path().resolve(), folder, script_name)
    input_dir = Path(script_dir, "input")
    output_dir = Path(script_dir, "output")
    input_file = Path(input_dir, "input.txt")

    return Locations(script_name, script_dir,
                     input_dir,
                     output_dir,
                     input_file)

### Retrieve the Input Data

This works by using your unique session cookie to retrieve your input data. E.g. from a URL like:

`https://adventofcode.com/2015/day/1/input`

In [ ]:
##################################################################
# Retrieving input data
##################################################################

def write_puzzle_input_file(year: int, day, locations: Locations):
    """ Use session key to obtain user's unique data for this year and day.
    Only retrieve if the input file does not already exist.
    Return True if successful.
    Requires env: AOC_SESSION_COOKIE, which can be set from the .env.
    """
    if os.path.exists(locations.input_file):
        logger.debug("%s already exists", os.path.basename(locations.input_file))
        return os.path.basename(locations.input_file)

    session_cookie = os.getenv('AOC_SESSION_COOKIE')
    if not session_cookie:
        raise ValueError("Could not retrieve session cookie.")

    logger.info('Session cookie retrieved: %s...%s', session_cookie[0:6], session_cookie[-6:])

    # Create input folder, if it doesn't exist
    if not locations.input_dir.exists():
        locations.input_dir.mkdir(parents=True, exist_ok=True)

    url = f"https://adventofcode.com/{year}/day/{day}/input"
    cookies = {"session": session_cookie}
    response = requests.get(url, cookies=cookies, timeout=5)

    data = ""
    if response.status_code == 200:
        data = response.text

        with open(locations.input_file, 'w') as file:
            logger.debug("Writing input file %s", os.path.basename(locations.input_file))
            file.write(data)
            return data
    else:
        raise ValueError(f"Unable to retrieve input data. HTTP response: {response.status_code}")


### Testing

A really simple function for testing that our solution produces the expected test output.

In [ ]:
def validate(test, answer):
    """
    Args:
        test: the answer given by our solution
        answer: the expected answer, e.g. from instructions
    """
    if test != answer:
        raise AssertionError(f"{test} != {answer}")

### Useful Helper Classes

In [ ]:
#################################################################
# POINTS, VECTORS AND GRIDS
#################################################################

@dataclass(frozen=True)
class Point:
    """ Class for storing a point x,y coordinate """
    x: int
    y: int

    def __add__(self, other: Point):
        return Point(self.x + other.x, self.y + other.y)

    def __mul__(self, other: Point):
        """ (x, y) * (a, b) = (xa, yb) """
        return Point(self.x * other.x, self.y * other.y)

    def __sub__(self, other: Point):
        return self + Point(-other.x, -other.y)

    def yield_neighbours(self, include_diagonals=True, include_self=False):
        """ Generator to yield neighbouring Points """

        deltas: list
        if not include_diagonals:
            deltas = [vector.value for vector in Vectors if abs(vector.value[0]) != abs(vector.value[1])]
        else:
            deltas = [vector.value for vector in Vectors]

        if include_self:
            deltas.append((0, 0))

        for delta in deltas:
            yield Point(self.x + delta[0], self.y + delta[1])

    def neighbours(self, include_diagonals=True, include_self=False) -> list[Point]:
        """ Return all the neighbours, with specified constraints.
        It wraps the generator with a list. """
        return list(self.yield_neighbours(include_diagonals, include_self))

    def get_specific_neighbours(self, directions: list[Vectors]) -> list[Point]:
        """ Get neighbours, given a specific list of allowed locations """
        return [(self + Point(*vector.value)) for vector in list(directions)]

    @staticmethod
    def manhattan_distance(a_point: Point) -> int:
        """ Return the Manhattan distance value of this vector """
        return sum(abs(coord) for coord in asdict(a_point).values())

    def manhattan_distance_from(self, other: Point) -> int:
        """ Manhattan distance between this Vector and another Vector """
        diff = self-other
        return Point.manhattan_distance(diff)

    def __repr__(self):
        return f"P({self.x},{self.y})"

class Vectors(Enum):
    """ Enumeration of 8 directions.
    Note: y axis increments in the North direction, i.e. N = (0, 1) """
    N = (0, 1)
    NE = (1, 1)
    E = (1, 0)
    SE = (1, -1)
    S = (0, -1)
    SW = (-1, -1)
    W = (-1, 0)
    NW = (-1, 1)

    @property
    def y_inverted(self):
        """ Return vector, but with y-axis inverted. I.e. N = (0, -1) """
        x, y = self.value
        return (x, -y)

class VectorDicts():
    """ Contains constants for Vectors """
    ARROWS = {
        '^': Vectors.N.value,
        '>': Vectors.E.value,
        'v': Vectors.S.value,
        '<': Vectors.W.value
    }

    DIRS = {
        'U': Vectors.N.value,
        'R': Vectors.E.value,
        'D': Vectors.S.value,
        'L': Vectors.W.value
    }

    NINE_BOX: dict[str, tuple[int, int]] = {
        # x, y vector for adjacent locations
        'tr': (1, 1),
        'mr': (1, 0),
        'br': (1, -1),
        'bm': (0, -1),
        'bl': (-1, -1),
        'ml': (-1, 0),
        'tl': (-1, 1),
        'tm': (0, 1)
    }

class Grid():
    """ 2D grid of point values. """
    def __init__(self, grid_array: list) -> None:
        self._array = grid_array
        self._width = len(self._array[0])
        self._height = len(self._array)

    def value_at_point(self, point: Point) -> int:
        """ The value at this point """
        return self._array[point.y][point.x]

    def set_value_at_point(self, point: Point, value: int):
        self._array[point.y][point.x] = value

    def valid_location(self, point: Point) -> bool:
        """ Check if a location is within the grid """
        if (0 <= point.x < self._width and  0 <= point.y < self._height):
            return True

        return False

    @property
    def width(self):
        """ Array width (cols) """
        return self._width

    @property
    def height(self):
        """ Array height (rows) """
        return self._height

    def all_points(self) -> list[Point]:
        points = [Point(x, y) for x in range(self.width) for y in range(self.height)]
        return points

    def rows_as_str(self):
        """ Return the grid """
        return ["".join(str(char) for char in row) for row in self._array]

    def cols_as_str(self):
        """ Render columns as str. Returns: list of str """
        cols_list = list(zip(*self._array))
        return ["".join(str(char) for char in col) for col in cols_list]

    def __repr__(self) -> str:
        return f"Grid(size={self.width}*{self.height})"

    def __str__(self) -> str:
        return "\n".join("".join(map(str, row)) for row in self._array)

### Useful Helper Functions

In [ ]:
#################################################################
# CONSOLE STUFF
#################################################################

def cls():
    """ Clear console """
    os.system('cls' if os.name=='nt' else 'clear')

#################################################################
# USEFUL FUNCTIONS
#################################################################

def binary_search(target, low:int, high:int, func, *func_args, reverse_search=False):
    """ Generic binary search function that takes a target to find,
    low and high values to start with, and a function to run, plus its args.
    Implicitly returns None if the search is exceeded. """

    res = None  # just set it to something that isn't the target
    candidate = 0  # initialise; we'll set it to the mid point in a second

    while low < high:  # search exceeded
        candidate = int((low+high) // 2)  # pick mid-point of our low and high
        res = func(candidate, *func_args) # run our function, whatever it is
        logger.debug("%d -> %d", candidate, res)
        if res == target:
            return candidate  # solution found

        comp = operator.lt if not reverse_search else operator.gt
        if comp(res, target):
            low = candidate
        else:
            high = candidate

def merge_intervals(intervals: list[list]) -> list[list]:
    """ Takes intervals in the form [[a, b][c, d][d, e]...]
    Intervals can overlap.  Compresses to minimum number of non-overlapping intervals. """
    intervals.sort()
    stack = []
    stack.append(intervals[0])

    for interval in intervals[1:]:
        # Check for overlapping interval
        if stack[-1][0] <= interval[0] <= stack[-1][-1]:
            stack[-1][-1] = max(stack[-1][-1], interval[-1])
        else:
            stack.append(interval)

    return stack

@cache
def get_factors(num: int) -> set[int]:
    """ Gets the factors for a given number. Returns a set[int] of factors.
        # E.g. when num=8, factors will be 1, 2, 4, 8 """
    factors = set()

    # Iterate from 1 to sqrt of 8,
    # since a larger factor of num must be a multiple of a smaller factor already checked
    for i in range(1, int(num**0.5) + 1):  # e.g. with num=8, this is range(1, 3)
        if num % i == 0: # if it is a factor, then dividing num by it will yield no remainder
            factors.add(i)  # e.g. 1, 2
            factors.add(num//i)  # i.e. 8//1 = 8, 8//2 = 4

    return factors

def to_base_n(number: int, base: int):
    """ Convert any integer number into a base-n string representation of that number.
    E.g. to_base_n(38, 5) = 123

    Args:
        number (int): The number to convert
        base (int): The base to apply

    Returns:
        [str]: The string representation of the number
    """
    ret_str = ""
    curr_num = number
    while curr_num:
        ret_str = str(curr_num % base) + ret_str
        curr_num //= base

    return ret_str if number > 0 else "0"


### Generic Initialisation


In [ ]:
FOLDER = "aoc"
YEAR = 2017
logger_identifier = "aoc" + str(YEAR)
logger = retrieve_console_logger(logger_identifier)
logger.setLevel(logging.DEBUG)

# Days

Here you'll find a template to build a solution for a given day, and then the solutions for all days in this event.

To copy the template day, select all the cells in the `Day n` template, add a new cell at the end, and then paste the cells there.

## Day n: title

In [ ]:
DAY = "n" # replace with actual number (without leading digit)
logger.setLevel(logging.DEBUG)
day_link = f"See [Day {DAY}](https://adventofcode.com/{YEAR}/day/{DAY})."
display(Markdown(day_link))

In [ ]:
d_name = "d" + str(DAY).zfill(2) # e.g. d01
script_name = "aoc" + str(YEAR) + d_name # e.g. aoc2017d01
locations = get_locations(d_name)

# SETUP LOGGING
logger.setLevel(logging.DEBUG)
# td.setup_file_logging(logger, locations.output_dir)

# Retrieve input and store in local file
try:
    write_puzzle_input_file(YEAR, DAY, locations)
except ValueError as e:
    logger.error(e)

with open(locations.input_file, mode="rt") as f:
    input_data = f.read().splitlines()

logger.debug("Input data:\n%s", input_data)

### Day n Part 1

Overview...

In [ ]:
def part1(data):
   pass

In [ ]:
%%time
validate(part1("abcdef"), "uvwxyz") # test with sample data
soln = part1(input_data)
logger.info(f"Part 1 soln={soln}")

### Day n Part 2

Overview...

In [ ]:
def part2(data):
    pass

In [ ]:
%%time
validate(part1("abcdef"), "uvwxyz") # test with sample data
soln = part2(input_data)
logger.info(f"Part 1 soln={soln}")

## Day 1: Inverse Captcha

In [ ]:
DAY = 1
day_link = f"See [Day {DAY}](https://adventofcode.com/{YEAR}/day/{DAY})."
display(Markdown(day_link))

In [ ]:
d_name = "d" + str(DAY).zfill(2) # e.g. d01
script_name = "aoc" + str(YEAR) + d_name # e.g. aoc2017d01
locations = get_locations(d_name)

# SETUP LOGGING
logger.setLevel(logging.INFO)
# td.setup_file_logging(logger, locations.output_dir)

# Retrieve input and store in local file
try:
    write_puzzle_input_file(YEAR, DAY, locations)
except ValueError as e:
    logger.error(e)

with open(locations.input_file, mode="rt") as f:
    input_data = f.read().strip()

logger.debug("Input data:\n%s", input_data)

### Day 1 Part 1

The time is 25ms to midnight, and we're inside the computer that prints the _Naughty or Nice_ list! Each day in this year's challenge brings us 1ms closer to midnight.

Today, we have to solve a Captcha to prove that we're _not_ human.

Sum all digits that match the next digit in a circular list.

In [ ]:
def sum_match_digits(data, offset: int) -> int:
    circular_digits = data + data
    logger.debug(circular_digits)
    total = 0
    for i in range(len(data)):
        if circular_digits[i] == circular_digits[i+offset]:
            total += int(circular_digits[i])

    return total

In [ ]:
def part1(data) -> int:
    return sum_match_digits(data, 1)

In [ ]:
%%time
validate(part1("91212129"), 9) # test with sample data
soln = part1(input_data)
logger.info("Part 1: total=%d", soln)
# logger.info("Execution time: %.3f seconds", t2 - t1)

### Day 1 Part 2

Sum all the digits that match a digit that is exactly halfway along.

In [ ]:
def part2(data) -> int:
    half = len(data)//2
    logger.debug("Half=%s", half)
    return sum_match_digits(data, half)

In [ ]:
%%time
validate(part2("12131415"), 4)
soln = part2(input_data)
logger.info("Part 2: total=%d", soln)

## Day 2: Corruption Checksum

In [ ]:
DAY = 2
day_link = f"See [Day {DAY}](https://adventofcode.com/{YEAR}/day/{DAY})."
display(Markdown(day_link))

In [ ]:
d_name = "d" + str(DAY).zfill(2) # e.g. d01
script_name = "aoc" + str(YEAR) + d_name # e.g. aoc2017d01
locations = get_locations(d_name)

# SETUP LOGGING
logger.setLevel(logging.INFO)
# td.setup_file_logging(logger, locations.output_dir)

# Retrieve input and store in local file
try:
    write_puzzle_input_file(YEAR, DAY, locations)
except ValueError as e:
    logger.error(e)

with open(locations.input_file, mode="rt") as f:
    input_data = f.read().splitlines()

logger.debug("Input data:\n%s", input_data)

### Day 2 Part 1

We neeed to calculate the spreadsheet's checksum.

For each row, determine the difference between the largest value and the smallest value; the checksum is the sum of all of these differences.

The input data is multiple lines.  The data can be split by either space or tab. I'll use regex to split on either. This returns a list of str values for each row.  Then I'll map these str values to int.

In [ ]:
def part1(data) -> int:
    """ Process each line. Get the largest and smallest int values from each line.
    Determine the difference.
    Sum the differences to give the checksum. """

    checksum = 0
    for row in data:
        vals = list(map(int, re.split(r'[\t ]+', row))) # split on either tab or space
        checksum += max(vals) - min(vals)

    return checksum

In [ ]:
%%time
validate(part1("""5 1 9 5
7 5 3
2 4 6 8""".splitlines()), 18) # test with sample data

soln = part1(input_data)
logger.info(f"Part 1 soln={soln}")

### Day 2 Part 2

The new goal is to find the only two numbers in each row where one evenly divides the other - that is, where the result of the division operation is a whole number. They would like you to find those numbers on each line, divide them, and add up each line's result.

Here I use `itertools.combinations` to return pairs of numbers from each row.

E.g.
[5, 9, 2, 8] -> (5, 9) (5, 2) (5, 8) (9, 2) (9, 8) (2, 8)

Note that reverse pairs are not included in `combinations`. If you want reverse pairs, use `permutations` instead.


In [ ]:
def part2(data) -> int:
    """ Process each line. Find the only two pairs of numbers where one is divisible by the other.
    Perform the division.
    Sum the quotients to give the checksum. """

    checksum = 0
    for row in data:
        vals = list(map(int, re.split(r'[\t ]+', row))) # split on either tab or space
        logger.debug(vals)
        for num1, num2 in combinations(vals, 2): # get all pairs of numbers, in one direction only
            if num1 % num2 == 0: # check if divisible
                checksum += num1 // num2
            elif num2 % num1 == 0: # check if divisible in opposite direction
                checksum += num2 // num1

    return checksum

In [ ]:
%%time
validate(part2("""5 9 2 8
9 4 7 3
3 8 6 5""".splitlines()), 9) # test with sample data

soln = part2(input_data)
logger.info(f"Part 2 soln={soln}")

## Day 3: Spiral Memory

In [ ]:
DAY = 3
day_link = f"See [Day {DAY}](https://adventofcode.com/{YEAR}/day/{DAY})."
display(Markdown(day_link))

In [ ]:
d_name = "d" + str(DAY).zfill(2) # e.g. d01
script_name = "aoc" + str(YEAR) + d_name # e.g. aoc2017d01
locations = get_locations(d_name)

# SETUP LOGGING
logger.setLevel(logging.INFO)
# td.setup_file_logging(logger, locations.output_dir)

# Retrieve input and store in local file
try:
    write_puzzle_input_file(YEAR, DAY, locations)
except ValueError as e:
    logger.error(e)

with open(locations.input_file, mode="rt") as f:
    input_data = int(f.read())

logger.debug("Input data:\n%d", input_data)

### Day 3 Part 1

We're presented with a 2D grid of spiral numbers, like this:

<pre>
 .   .   .   .   .   .   .
 .  17  16  15  14  13   .
 .  18   5   4   3  12   .
 .  19   6   <b>1</b>   <b>2</b>  11  28
 .  20   7   8   9  <b>10</b>  27
 .  21  22  23  24  25  <b>26</b>
 .   .   .   .   .   .   .
</pre>

Requested data can only be retrieved from square `1`, and programs can only move U, D, L, R. They always take the shortest path using Manhattan distance. I've also shown the location of the lowest value in each successive square.

**How many steps are required to carry the data from the square identified in your puzzle input all the way to the access port?**

#### Requirement

- We need to determine the location of our input data.
- Then determine Manhattan distance of this value to our `1` position.

#### Options

1. We could use a generator to allocate each successive value.
2. We could determine the size of each successive square. And then determine the position of our data in that outermost square.

I'm going to with option 2, as it should be pretty quick.

#### Determine the Perimeter

- Any given perimeter is given by: $p = 4(e-1)$, where e is the length of the edge. Or: $p = 4e-4$
- The length of the edge is given by: $e = 2(r+1)-1 = 2r + 1$, where r is the current ring.
- So, $p = 4(2r+1-1) = 8r$

In [ ]:
def perimeter(ring: int) -> int:
    """ Return the total number of values in this particular ring.
    I.e. successive squares have edges of length 1, 3, 5, 7, etc. But we won't count the starting square as a ring.
    Perimeters will be 8, 16, 24, etc """
    return (8 * ring)

def position(target: int, ring: int, ring_start: int) -> Point:
    """ Determine the location of our target value in the grid.
    Do this by starting in our lower right position, and then move one position at a time, until we reach our target.
    Args:
    - target = the value we need to know the position of
    - ring = the current ring (where 1 is the centre, 2 is the second ring, etc)
    - ring_start = the lowest value of this ring
    """
    logger.debug(f"Looking for {target} in ring {ring}, which starts at {ring_start}")

    # the inclusive boundaries of our ring
    x_max = y_max = 0 + ring
    x_min = y_min = 0 - ring

    curr_val = ring_start
    curr_x = 0 + ring
    curr_y = 0 + (ring-1)

    if ring_start == target:
        return Point(curr_x, curr_y)

    assert curr_val < target and curr_x == x_max, "We're on the right edge"
    
    # move up until we can't go further
    while curr_y > y_min:
        curr_val += 1
        curr_y -= 1

        if curr_val == target:
            return Point(curr_x, curr_y)

    assert curr_val < target and curr_y == y_min, "We're on the top row"

    # move left until we can't go further
    while curr_x > x_min:
        curr_val += 1
        curr_x -= 1

        if curr_val == target:
            return Point(curr_x, curr_y)

    assert curr_val < target and curr_x == x_min, "We're on the left edge"
        
    # move down until we can't go further
    while curr_y < y_max:
        curr_val += 1
        curr_y += 1

        if curr_val == target:
            return Point(curr_x, curr_y)

    assert curr_val < target and curr_y == y_max, "We're on the bottom row"

    # move right until we can't go further
    while curr_x < x_max:
        curr_val += 1
        curr_x += 1

        if curr_val == target:
            return Point(curr_x, curr_y)

    assert False, "We can't be here!"

In [ ]:
def part1(data: int) -> int:

    # Get the ring where our data is
    prev_highest = highest = 1 # we start with 1 at the center
    ring = 0 
    while highest <= data:
        ring += 1
        per = perimeter(ring)
        prev_highest = highest
        highest += per

    # Now get the position of our data in the ring
    data_point = position(data, ring, prev_highest+1)
    logger.debug(f"Our data is at {data_point}")
    dist = data_point.manhattan_distance_from(Point(0,0))
    logger.debug(f"Manhattan distance = {dist}")
    return dist

In [ ]:
%%time
validate(part1(1024), 31) # test data
soln = part1(input_data)
logger.info(f"Part 1 soln={soln}")

### Day 3 Part 2

Darn, I should have gone with Option 1!!

The puzzle has changed such that each value assigned is the sum of the adjacent values that have already been assigned. 

**What is the first value written that is larger than your puzzle input?**

<pre>
147  142  133  122   59
304    5    4    2   57
330   10    <b>1</b>    <b>1</b>   54
351   11   23   25   <b>26</b>
362  747    .    .    .
</pre>

So, we're going to have to generate each value succcessively.  The number we need to reach isn't very high, so this won't be a problem.

This feels like a perfect time to use a NumPy array!

- Create a generator that returns successive positions, spiralling out from the centre.
- Guess at a starting size for a square array, and initialise it with zeroes.
- Work out the middle of our array, and save this is a delta that we will add to EVERY
  coordinate returned by our spiral generator. (Because our spiral generator will start with 0,0.)
- Initialise our middle coordinate to 1.
- Then simply use NumPy sum to calculate the sum of all adjacent locations. Remember that those we haven't filled yet will have a value of 0.
- Add this calculated sum into the new location.

Easy!

In [ ]:
def spiral_next_posn():
    """A generator that yields the next location in the spiral """
    curr_x = curr_y = 0
    yield curr_x, curr_y # the origin

    ring = 1
    while True:
        # the inclusive boundaries of our ring
        x_max = y_max = 0 + ring
        x_min = y_min = 0 - ring

        # our lower right starting position in the new ring
        curr_x = 0 + ring
        curr_y = 0 + (ring-1)
        yield curr_x, curr_y        
        
        while curr_y > y_min: # we're on the right edge
            curr_y -= 1  # move up
            yield curr_x, curr_y

        while curr_x > x_min: # we're on the top edge
            curr_x -= 1  # move left
            yield curr_x, curr_y

        while curr_y < y_max: # we've on the left edge
            curr_y += 1  # move down
            yield curr_x, curr_y

        while curr_x < x_max: # we're on the bottom edge
            curr_x += 1  # move right
            yield curr_x, curr_y

        ring += 1 # move to next ring and start again

In [ ]:
def part2(data: int) -> int:
    """ We want this to return the first value larger than our input """
    
    size = 20
    grid = np.zeros((size, size), dtype=np.int32)
    
    spiral_generator = spiral_next_posn()
    # this returns 0,0
    # but we want this to be the centre of our grid. So we'll add a delta
    delta = size // 2
    
    x, y = next(spiral_generator) 
    x, y = x+delta, y+delta
    curr_val = 1
    grid[y, x] = curr_val # initialise the center
    logger.debug("Posn 1: %d, %d has value %d", x, y, curr_val)    

    # now we continue around the spiral
    for i, posn in enumerate(spiral_generator, start=2):
        x, y = posn
        x, y = x+delta, y+delta
        assert x<size and y<size, "No solution in grid of this size"
        
        curr_val = int(grid[y-1:y+2, x-1:x+2].sum()) # Get the sum of all adjacent values
        grid[y, x] = curr_val

        logger.debug(f"Posn %d: %d, %d has value %d", i, x, y, curr_val)
        if curr_val > data:
            break

    return curr_val

In [ ]:
%%time
validate(part2(142), 147) # assert that 147 comes after 142
soln = part2(input_data)
logger.info(f"Part 2 soln={soln}")